```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de datos no estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2021
Profesor: Facundo Santiago
```

# Solución: ¿Qué tan memorable es un video?

Este notebook les permitirá tener a disposición todos los recursos que se vieron en durante la materia. Todos los fragmentos de código se descargaran utilizando la sección preparación del ambiente.

## Preparación del ambiente

### Sets de datos

In [9]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/ground_truth.csv --directory-prefix ./Data/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/audio_vectors.csv --directory-prefix ./Data/Features/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/c3d_vectors.csv --directory-prefix ./Data/Features/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/caption_features.csv --directory-prefix ./Data/Features/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/colors_features.csv --directory-prefix ./Data/Features/ --quiet

### Librerías de codigo que se utilizan durante el curso

In [ ]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/solucion_base.txt --quiet
!pip install -r solucion_base.txt --quiet

#### Audio

In [10]:
!wget -N https://storage.googleapis.com/audioset/yamnet.h5 --directory-prefix ./Models/yamnet/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/Models/yamnet/yamnet_class_map.csv --directory-prefix ./Models/yamnet/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/Utils/audio_plotter.py --directory-prefix ./Utils/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/yamnet/yamnet.py --directory-prefix ./yamnet/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/yamnet/params.py --directory-prefix ./yamnet/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Audio/yamnet/features_lib.py --directory-prefix ./yamnet/ --quiet

#### NLP

In [11]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextDataset.py --directory-prefix ./Utils/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/TextNormalizer.py --directory-prefix ./Utils/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/PadSequenceTransformer.py --directory-prefix ./Utils/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Utils/Word2VecVectorizer.py --directory-prefix ./Utils/ --quiet
!wget https://santiagxf.blob.core.windows.net/public/Word2Vec/model-en.bin --directory-prefix ./Models/Word2Vec --quiet

#### Vision

In [12]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Vision/Utils/Cognitive.py --directory-prefix ./Utils/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Vision/Utils/Plotting.py --directory-prefix ./Utils/ --quiet

### Imports

In [8]:
import pandas as pd
import numpy as np
import librosa
import os
import soundfile as sf
import torch
import transformers
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, SpatialDropout1D
import tensorflow_datasets as tfds

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from yamnet.yamnet import yamnet_frames_model
import yamnet.yamnet as yamnet_model
from yamnet.params import Params

from Utils.audio_plotter import plot_audio_embeddings
from Utils.TextDataset import ClassificationDataset
from Utils.TextNormalizer import TweetTextNormalizer
from Utils.Word2VecVectorizer import Word2VecVectorizer
from Utils.PadSequenceTransformer import PadSequenceTransformer

import azure.cognitiveservices.speech as speechsdk
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

## Solución

Cuentan con 2 tipos de anotaciones para cada uno de los fragmentos de video disponibles:
 - **memorability_score:** Representa el puntaje de memorabilidad de la secuencia en particular, desde 0 a 1. Valores más grandes son mejores.
 - **memorable:** Variable categórica que representa si un video es memorable o no. Un video con `memorability_score` superior a `0.5` es marcado como memorable (`1`), sino es marcado como no memorable (`0`)

In [ ]:
labels = pd.read_csv('Data/ground_truth.csv')

In [ ]:
labels.head(2)

,movie_name,start(sec),end(sec),sequence_name,Neutral (1)_Typical (0),nb_annotations,memorability_score,memorabable
0,127 hours,2000,2010,127_hours_2000_2010_1,0,5,1.0,1
1,127 hours,2182,2192,127_hours_2182_2192_5,1,8,0.0,0


Las siguientes variables contienen la información preprocesada:

In [ ]:
captions = pd.read_csv('Data/Features/caption_features.csv')
colors = pd.read_csv('Data/Features/colors_features.csv')
audio_embeddings = pd.read_csv('Data/Features/audio_vectors.csv').to_numpy()[:,2:].reshape((-1,20,1024))
video_embeddings = pd.read_csv('Data/Features/c3d_vectors.csv')

> **Importante:** Note que `audio_embeddings` es un tensor y no un `pd.DataFrame`. Esto se debe a que las dimensiones de este tipo de dato son multidimensionales y los dataframes en `pandas` no permiten este tipo de datos.

In [ ]:
# completar solución